# Import

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv

from CMEP import CMEP_Atom
from qsps import QSP_HNC
from constants import *
# from hnc_Ng import  HNC_solver
# from qsps import *

prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
        
        
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

Ha_to_J = 4.359744e-18
k_B = 1.380649e-23 
m_p = 1.672621898e-27 # kg
eV_to_K = 11604.5250061598
J_to_erg = 1e7
Ha_to_erg = Ha_to_J*J_to_erg

# Function that sets up HNC QSP 

## Load TCCW Cases

In [17]:
element='He'

if element=='Xe':
    ni_cc = 1.25e20
    Te_eV = 16.64e3/eV_to_K
    Ti_eV = 10.47e3/eV_to_K
    Z, A = 54, 131

elif element=='He':
    ni_cc = 1.83e21 # 1/m^3
    Te_eV = 14.48e3/eV_to_K
    Ti_eV = 11.66e3/eV_to_K
    Z, A = 2,4

ni_AU = ni_cc*1e6*aB**3

r_s_AU = QSP_HNC.rs_from_n(ni_AU)
r_c = 0
if r_c>1:
    r_c=1e-5
print("Te = {0:.3e} eV, n_i = {1:.3e} 1/cc".format(Te_eV, ni_cc))
print("Te = {0:.3e} K, ρ = {1:.3e} [g/cc] ".format(Te_eV*eV_to_K, 1e3*ni_cc*m_p*A ))
print("r_s = {0:.3f}, r_c/r_s = {1:.3f}".format(r_s_AU, r_c))

Te = 1.248e+00 eV, n_i = 1.830e+21 1/cc
Te = 1.448e+04 K, ρ = 1.224e-02 [g/cc] 
r_s = 9.584, r_c/r_s = 0.000


In [28]:
atom = CMEP_Atom(Z, A, ni_cc, Ti_eV, Te_eV, 
                 βu_options = {'add_bridge':True, 'pseudopotential':True}, 
                 qsp_options= {'r_c':0.1},
                 hnc_options= {'kappa_multiscale':10, 'R_max':3,'N_bins':500},
                 hnc_solve_options={'alpha_Picard':0.1})

Te_in_eV: 1.248
Ti_in_eV: 1.005
Te: 1.248 eV, Zbar = 0.462


In [29]:
atom.run_hnc()
atom.get_cv(ε_derivative=1e-7, ε_table=1e-6)

0: Initial condition Total Error: 1.299e+00
Exiting status 0, reverting to best index so far: 141
Final iter:143, Total Error: 3.404e-07
Te: 1.248 eV, Zbar = 0.462
0: Initial condition Total Error: 1.017e-06
Exiting status 0, reverting to best index so far: 0
Final iter:2, Total Error: 9.319e-07
Te: 1.248 eV, Zbar = 0.462
0: Initial condition Total Error: 1.099e-06
Exiting status 0, reverting to best index so far: 0
Final iter:2, Total Error: 1.022e-06
Te: 1.248 eV, Zbar = 0.462
0: Initial condition Total Error: 7.960e-07
Exiting status 0, reverting to best index so far: 0
Final iter:2, Total Error: 7.408e-07
Te: 1.248 eV, Zbar = 0.462
0: Initial condition Total Error: 1.483e-06
Exiting status 0, reverting to best index so far: 1
Final iter:3, Total Error: 1.226e-06

_____________________________
Heat Capacity Results 
c^e_v = 3.261e+03 [J/m^3/K], c^i_v  = 3.512e+04 [J/m^3/K] 
c_v = 3.838e+04 [J/m^3/K] 

c^e_v/(ne kB)= 0.279 , c^i_v/(ni kB)   = 1.390 
c_v/(n kB) = 1.039 


In [ ]:
# r_c = 0.1
Heat Capacity Results 
c^e_v = 3.261e+03 [J/m^3/K], c^i_v  = 3.512e+04 [J/m^3/K] 
c_v = 3.838e+04 [J/m^3/K] 

c^e_v/(ne kB)= 0.279 , c^i_v/(ni kB)   = 1.390 
c_v/(n kB) = 1.039  

In [ ]:
# r_c = 0
Heat Capacity Results 
c^e_v = 1.864e+03 [J/m^3/K], c^i_v  = 3.508e+04 [J/m^3/K] 
c_v = 3.695e+04 [J/m^3/K] 

c^e_v/(ne kB)= 0.160 , c^i_v/(ni kB)   = 1.389 
c_v/(n kB) = 1.000 

In [6]:
atom.get_effective_ion_cp()

0: Initial condition Total Error: 2.440e-03
Exiting status 0, reverting to best index so far: 6
Final iter:8, Total Error: 5.909e-09
0: Initial condition Total Error: 4.382e-03
Exiting status 0, reverting to best index so far: 8
Final iter:10, Total Error: 6.985e-09
0: Initial condition Total Error: 4.456e-03
Exiting status 0, reverting to best index so far: 8
Final iter:10, Total Error: 8.409e-09
0: Initial condition Total Error: 2.368e-03
Exiting status 0, reverting to best index so far: 6
Final iter:8, Total Error: 5.476e-09

_____________________________
Heat Capacity Results (Effective Ion Picture) 
c_p = 3.103e+04 [J/m^3/K] = 3.103e+05 [erg/cc/K]
c_p^ideal = 2.071e+04 [J/m^3/K] = 2.071e+05 [erg/cc/K]
c_v = 1.801e+04 [J/m^3/K] = 1.801e+05 [erg/cc/K]

γ = cp/cv = 1.723e+00

E/nkBT = 2.027, P/nkBT = 1.637 
cp/nkB = 3.746, cv/nkB = 2.174 
Total cv/nkB estimate (add ideal electrons):
c_v_tot_estimate = 1.837
